# helpers

> State getters for the review step from InteractionContext

In [ ]:
#| default_exp components.helpers

In [ ]:
#| export
from typing import Any, Dict, List, Optional

from cjm_fasthtml_interactions.core.context import InteractionContext

from cjm_transcript_segmentation.models import TextSegment
from cjm_transcript_vad_align.models import VADChunk

from cjm_transcript_review.models import ReviewStepState

## State Getters

These extract review state from the nested `step_states.review` path in the
interaction context. The review step also reads segments and VAD chunks from
the segmentation and alignment steps respectively.

In [ ]:
#| export
def _get_review_state(
    ctx:InteractionContext  # Interaction context with state
) -> ReviewStepState:  # Typed review step state
    """Get the full review step state from context."""
    step_states = ctx.state.get("step_states", {})
    return step_states.get("review", {})

def _get_focused_index(
    ctx:InteractionContext,  # Interaction context with state
    default:int=0,  # Default focused index
) -> int:  # Currently focused segment index
    """Get the currently focused segment index."""
    state = _get_review_state(ctx)
    return state.get("focused_index", default)

def _get_visible_count(
    ctx:InteractionContext,  # Interaction context with state
    default:int=5,  # Default visible card count
) -> int:  # Number of visible cards in viewport
    """Get the stored visible card count."""
    state = _get_review_state(ctx)
    return state.get("visible_count", default)

def _get_is_auto_mode(
    ctx:InteractionContext,  # Interaction context with state
) -> bool:  # Whether card count is in auto-adjust mode
    """Get whether the card count is in auto-adjust mode."""
    state = _get_review_state(ctx)
    return state.get("is_auto_mode", False)

def _get_card_width(
    ctx:InteractionContext,  # Interaction context with state
    default:int=50,  # Default card width in rem
) -> int:  # Card stack width in rem
    """Get the stored card stack width."""
    state = _get_review_state(ctx)
    return state.get("card_width", default)

## Cross-Step Data Access

Review reads segments and VAD chunks from their respective step states.

In [ ]:
#| export
def _get_segments(
    ctx:InteractionContext  # Interaction context with state
) -> List[TextSegment]:  # List of TextSegment objects from segmentation step
    """Get text segments from segmentation step state."""
    step_states = ctx.state.get("step_states", {})
    seg_state = step_states.get("segmentation", {})
    segment_dicts = seg_state.get("segments", [])
    return [TextSegment.from_dict(s) for s in segment_dicts]

def _get_vad_chunks(
    ctx:InteractionContext  # Interaction context with state
) -> List[VADChunk]:  # List of VADChunk objects from alignment step
    """Get VAD chunks from alignment step state."""
    step_states = ctx.state.get("step_states", {})
    align_state = step_states.get("alignment", {})
    chunk_dicts = align_state.get("vad_chunks", [])
    return [VADChunk.from_dict(c) for c in chunk_dicts]

def _get_media_path(
    ctx:InteractionContext  # Interaction context with state
) -> Optional[str]:  # Path to original audio file or None
    """Get the original audio file path from alignment step."""
    step_states = ctx.state.get("step_states", {})
    align_state = step_states.get("alignment", {})
    return align_state.get("media_path")

def _is_aligned(
    ctx:InteractionContext  # Interaction context with state
) -> bool:  # True if segment count matches VAD chunk count
    """Check if segments are aligned with VAD chunks (1:1 match)."""
    segments = _get_segments(ctx)
    vad_chunks = _get_vad_chunks(ctx)
    return len(segments) == len(vad_chunks) and len(segments) > 0

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()